In [2]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to tensors
])

# Load dataset
train_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\faculity project\processed resized data\augmented ali data\balanced_dataset\train", transform=transform)
val_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\faculity project\processed resized data\augmented ali data\balanced_dataset\valid", transform=transform)

# Create data loaders
train_loader = DataLoader(train_size, batch_size=26, shuffle=True)
val_loader = DataLoader(val_size, batch_size=26, shuffle=False)

# Load pre-trained ResNet101V2 model
base_model = torchvision.models.resnet101(pretrained=True)

# # Freeze the parameters of the pretrained model
# for param in base_model.parameters():
#     param.requires_grad = False

# Get all layers of the ResNet-101 model except the last one
all_but_last = nn.Sequential(*list(base_model.children())[:-1])

num_classes = len(train_size.classes)  # Number of classes in your dataset

# Define the extended model with additional layers
model = nn.Sequential(
    all_but_last,
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.3),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(1024, num_classes),
    nn.Softmax(dim=1)
)

# Optionally, move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 50

best_val_loss = float('inf')
patience = 5
counter = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True) as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            t.set_postfix(loss=running_loss / total, accuracy=100 * correct / total)
            
    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    # Calculate average validation loss and accuracy
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * val_correct / val_total

    print(f'Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model_paper.pth')
    else:
        counter += 1

    if counter >= patience:
        print(f'Validation loss did not improve for {patience} consecutive epochs. Early stopping...')
        break

print('Training completed!')


Epoch 1/50: 100%|██████████████████████████████████████| 1847/1847 [18:53<00:00,  1.63it/s, accuracy=89.3, loss=0.0666]


Validation Loss: 1.6462, Accuracy: 97.48%


Epoch 2/50: 100%|██████████████████████████████████████| 1847/1847 [18:38<00:00,  1.65it/s, accuracy=94.8, loss=0.0643]


Validation Loss: 1.6572, Accuracy: 96.27%


Epoch 3/50: 100%|██████████████████████████████████████| 1847/1847 [18:44<00:00,  1.64it/s, accuracy=95.8, loss=0.0639]


...

Validation Loss: 1.6897, Accuracy: 92.86%


Epoch 17/50: 100%|█████████████████████████████████████| 1847/1847 [18:31<00:00,  1.66it/s, accuracy=98.5, loss=0.0629]


Validation Loss: 1.6281, Accuracy: 99.05%


Epoch 18/50: 100%|█████████████████████████████████████| 1847/1847 [18:31<00:00,  1.66it/s, accuracy=98.7, loss=0.0628]


Validation Loss: 1.6311, Accuracy: 98.77%
Validation loss did not improve for 5 consecutive epochs. Early stopping...
Training completed!


In [4]:
import torch
import torchvision
from thop import profile
from torchvision import transforms
import torch.nn as nn
from torchvision import datasets, transforms
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

# Load pre-trained ResNet101V2 model
base_model = torchvision.models.resnet101(pretrained=False)

# # Freeze the parameters of the pretrained model
# for param in base_model.parameters():
#     param.requires_grad = False

# Get all layers of the ResNet-101 model except the last one
all_but_last = nn.Sequential(*list(base_model.children())[:-1])

num_classes = 11  # Number of classes in your dataset

# Define the extended model with additional layers
model = nn.Sequential(
    all_but_last,
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.3),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(1024, num_classes),
    nn.Softmax(dim=1)
)


#load our trained model
model.load_state_dict(torch.load('best_model.pth'))

# Define input tensor shape (batch_size, channels, height, width)
input_shape = (1, 3, 224, 224)  # Assuming input images are RGB with size 224x224

# Move the model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Generate random input data for profiling
inputs = torch.randn(input_shape).to(device)

# Calculate FLOPS
flops, params = profile(model, inputs=(inputs,), verbose=False)

# Calculate number of trainable parameters
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("For Modified:")
print(f"FLOPS: {flops / (10**6):.2f} million")
print(f"Number of parameters: {params / (10**6):.2f} million")
print(f"Number of trainable parameters: {num_trainable_params / (10**6):.2f} million")



For Modified:
FLOPS: 7866.50 million
Number of parameters: 44.61 million
Number of trainable parameters: 44.61 million


In [40]:
import torch
import torchvision
from thop import profile
from torchvision import transforms
import torch.nn as nn
from torchvision import datasets, transforms
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import torchvision.models as models

# Load ResNet101V2 model
model = models.resnet101(pretrained=False)

# Modify the last fully connected layer to classify 7 classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 15)

# Count the number of parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total number of parameters: {total_params / 10**6:.2f} million")
print(f"Number of trainable parameters: {trainable_params / 10**6:.2f} million")


# Create a dummy input
dummy_input = torch.randn(1, 3, 224, 224)

# Calculate FLOPS
flops, _ = profile(model, inputs=(dummy_input,), verbose = False)


print(f"FLOPS: {flops / 10**6:.2f} million")

Total number of parameters: 42.51 million
Number of trainable parameters: 42.51 million
FLOPS: 7864.40 million


In [10]:
# Get class labels
class_labels = train_size.classes

# Print class labels with their index
for i, label in enumerate(class_labels):
    print(f"Index {i}: Class Label - {label}")

Index 0: Class Label - Common wheat
Index 1: Class Label - Convolvulus arvensis
Index 2: Class Label - Cotton
Index 3: Class Label - Euphorbia peplus
Index 4: Class Label - Grass
Index 5: Class Label - Lolium multiflorum
Index 6: Class Label - Maize
Index 7: Class Label - Nutgrass
Index 8: Class Label - Purslane
Index 9: Class Label - Sesame
Index 10: Class Label - Sugar beet
Index 11: Class Label - Tomato


In [12]:
#to test each class prediction accuracy

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Convert images to tensors
])

class_labels = train_size.classes

# Load dataset
val_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\faculity project\test\New folder", transform=transform)

# Create data loader for validation set
val_loader = DataLoader(val_size, batch_size=32, shuffle=False)

# Load pre-trained ResNet101V2 model
base_model = torchvision.models.resnet101(pretrained=False)

# # Freeze the parameters of the pretrained model
# for param in base_model.parameters():
#     param.requires_grad = False

# Get all layers of the ResNet-101 model except the last one
all_but_last = nn.Sequential(*list(base_model.children())[:-1])

num_classes = 12  # Number of classes in your dataset

# Define the extended model with additional layers
model = nn.Sequential(
    all_but_last,
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.3),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(1024, num_classes),
    nn.Softmax(dim=1)
)


#load our trained model
model.load_state_dict(torch.load('best_model_paper.pth'))

# Optionally, move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

# Evaluate the model on the validation dataset
model.eval()
class_correct = [0] * num_classes
class_total = [0] * num_classes

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for i in range(len(labels)):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                class_correct[label] += 1
            class_total[label] += 1

# Calculate accuracy for each class
class_accuracy = [class_correct[i] / class_total[i] if class_total[i] != 0 else 0 for i in range(num_classes)]
for i in range(num_classes):
    print(f"Accuracy for class {class_labels[i]}: {class_accuracy[i] * 100:.2f}%")


Evaluating: 100%|████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.71it/s]

Accuracy for class Common wheat: 100.00%
Accuracy for class Convolvulus arvensis: 100.00%
Accuracy for class Cotton: 25.00%
Accuracy for class Euphorbia peplus: 14.29%
Accuracy for class Grass: 100.00%
Accuracy for class Lolium multiflorum: 100.00%
Accuracy for class Maize: 50.00%
Accuracy for class Nutgrass: 96.43%
Accuracy for class Purslane: 100.00%
Accuracy for class Sesame: 100.00%
Accuracy for class Sugar beet: 25.00%
Accuracy for class Tomato: 0.00%


In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Define lists to store metrics
class_precision = []
class_recall = []
class_f1_score = []

# Evaluate the model on the validation dataset
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate precision, recall, and F1-score for each class
for i in range(num_classes):
    precision = precision_score(true_labels, predictions, labels=[i], average='micro')
    recall = recall_score(true_labels, predictions, labels=[i], average='micro')
    f1 = f1_score(true_labels, predictions, labels=[i], average='micro')
    class_precision.append(precision)
    class_recall.append(recall)
    class_f1_score.append(f1)
    print(f"Metrics for class {class_labels[i]}:")
    print(f"  Precision: {precision:.2f}")
    print(f"  Recall: {recall:.2f}")
    print(f"  F1-score: {f1:.2f}")


Evaluating: 100%|████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.92it/s]

Metrics for class Common wheat:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Convolvulus arvensis:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Cotton:
  Precision: 1.00
  Recall: 0.25
  F1-score: 0.40
Metrics for class Euphorbia peplus:
  Precision: 0.67
  Recall: 0.14
  F1-score: 0.24
Metrics for class Grass:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Lolium multiflorum:
  Precision: 0.97
  Recall: 1.00
  F1-score: 0.98
Metrics for class Maize:
  Precision: 0.67
  Recall: 0.50
  F1-score: 0.57
Metrics for class Nutgrass:
  Precision: 1.00
  Recall: 0.96
  F1-score: 0.98
Metrics for class Purslane:
  Precision: 0.33
  Recall: 1.00
  F1-score: 0.50
Metrics for class Sesame:
  Precision: 0.64
  Recall: 1.00
  F1-score: 0.78
Metrics for class Sugar beet:
  Precision: 0.11
  Recall: 0.25
  F1-score: 0.15
Metrics for class Tomato:
  Precision: 0.00
  Recall: 0.00
  F1-score: 0.00
